In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib qt5

# Model configuration

<img src='image/model_tree.png' width='400' alt='pk tree' align='center' />


Refer to **"Integrated surface/subsurface permafrost thermal hydrology: Model formulation and proof‐of‐concept simulations" (Painter et al., 2016)** 

# Saturation Patition

* Three phases: gas, liquid water, ice.
* Saturations are determined by temperature and capillary pressue between phases.
* The pressure gradient between liquid water and ice (capillary pressure between ice and liquid) draws liquid water toward the freezing front in porous media during freezing period (**cryosuction**).
* Saturation model [refer to Painter & Karra (2014)]:

<img src='image/sat_math_declar.png' width='600' alt='saturation model' align='center' />

<img src='image/sat_xml.png' width='800' alt='pk tree' align='center' />

# Relative permeability

## ATS options

<img src='image/kr_options.png' width='900' alt='pk tree' align='center' />

Normally used:
* water retention model + van Genuchten - Mualem
* freezing Brooks-Corey + Brooks-Corey
* SutraICE + Brooks-Corey

## Kr effect

In [74]:
df_wrm_vg = pd.read_csv("hillslope-wrmVG_krWRMtypeVG/observation.dat", comment='#', delimiter=' ').iloc[365:].reset_index(drop=True)
df_frzbc_bc = pd.read_csv("hillslope-wrmVG_krFrzBCtypeBC/observation.dat", comment='#', delimiter=' ').iloc[365:].reset_index(drop=True)
df_sutraice_bc = pd.read_csv("hillslope-wrmVG_krSutraICEtypeBC/observation.dat", comment='#', delimiter=' ').iloc[365:].reset_index(drop=True)


In [75]:
fig, ax = plt.subplots(1,1, figsize=(10,4))
plt.subplots_adjust(left=0.06, right=0.94, top=0.98, bottom=0.11)
density = 55500
key = 'discharge [mol/d]'
(df_wrm_vg[key]/density).plot(ax=ax, label='water retention model, van Genuchten-Mualem')
(df_frzbc_bc[key]/density).plot(ax=ax, label='freezing Brooks-Corey, Brooks-Corey')
(df_sutraice_bc[key]/density).plot(ax=ax, label='SutraICE, Brooks-Corey')

ax2 = ax.twinx()
df_wrm_vg['rain [m3/d]'].plot(ax=ax2, color='grey')
ax2.invert_yaxis()

ax.set_xlabel('Days')
ax.set_ylabel(r'Discharge [m$^3$/d]');
ax2.set_ylabel(r'Rainfall [m$^3$/d]');

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=labels, loc='center');

In [79]:
fig, ax = plt.subplots(1,1, figsize=(10,4))
plt.subplots_adjust(left=0.06, right=0.94, top=0.98, bottom=0.11)
density = 55500
key = 'surface-subsurface exchange [mol/d]'
(-df_wrm_vg[key]/density).plot(ax=ax, label='water retention model, van Genuchten-Mualem')
(-df_frzbc_bc[key]/density).plot(ax=ax, label='freezing Brooks-Corey, Brooks-Corey')
(-df_sutraice_bc[key]/density).plot(ax=ax, label='SutraICE, Brooks-Corey')


ax.set_xlabel('Days')
ax.set_ylabel(r'Infiltration [m$^3$/d]');

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=labels, loc='upper center');

<img src='image/sat180d.png' width='1200' alt='pk tree' align='center' />

## Implementation in input files

<img src='image/wrmvg.png' width='700' alt='pk tree' align='center' />

<img src='image/krzbc.png' width='700' alt='pk tree' align='center' />